In [1]:
import docx2txt
from book_builder.book_builder import BookBuilder

In [2]:
import sqlite3
conn = sqlite3.connect('data/kcbible.db')
db = conn.cursor()

In [3]:
johan_txt = docx2txt.process('docx_normalized/Johan.docx')
lines = [line.strip() for line in johan_txt.split('\n\n') if line.strip() != '']
bb = BookBuilder(lines[0])
for line in lines[1:]:
    bb.process(line)
book = bb.build()
print(f"book {book.title} has {len(book.chapters)} chapters with a combined total of {sum([len(c.verses) for c in book.chapters])} verses")

book JOHAN has 21 chapters with a combined total of 878 verses


In [8]:
book_id_lookup = {'Johan': 1}

In [ ]:
db.execute('insert into book(name, language) values(?,?)',('Johan','Kaang Chin'))
book_id_lookup['Johan'] = db.lastrowid
conn.commit()

In [5]:
insert_chapter = """
    insert into chapter(number, title, book_id)
    values(?,?,?)
    """

In [6]:
insert_verse = """
    insert into verse(number, title, text, chapter_id)
    values(?,?,?,?)
"""

In [9]:
for c in book.chapters:
    db.execute(insert_chapter, (c.number, c.title if c.title else None, book_id_lookup['Johan']))
    chapter_id = db.lastrowid
    for v in c.verses:
        db.executemany(insert_verse, [(v.number,v.title if v.title else None,v.text,chapter_id)])
conn.commit()

In [10]:
conn.close()